In [ ]:
# default_exp car

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Car RL

In [ ]:
from bfh_mt_hs2020_rl_basics.env import CarEnv
from bfh_mt_hs2020_rl_basics.agent import Agent
from bfh_mt_hs2020_rl_basics.bridge import Bridge
from bfh_mt_hs2020_rl_basics.loop import LoopControl

import gym
from gym.spaces import Tuple, Discrete, Box
import numpy as np
import warnings

from types import SimpleNamespace

import torch
from torch.optim import Optimizer, Adam

In [ ]:
HYPERPARAMS = {
    'base_setup': SimpleNamespace(**{
            # agent
            'agent_device'                : "cpu",    # cpu or cuda
            'agent_gamma_exp'             : 0.9,      # discount_factor for experience_first_last.. shouldn't matter since step_size is only 1
            'agent_buffer_size'           : 1000,     # size of replay buffer
            'agent_eps_start'             : 1.0,      # epsilon start
            'agent_eps_final'             : 0.02,     # epsilon end
            'agent_eps_frames'            : 10**5,    # epsilon frames -> how many frames until 0.02 should be reached .. decay is linear
            'agent_target_net_sync'       : 1000,     # sync TargetNet with weights of DNN every .. iterations
  
            # bridge  
            'bridge_optimizer'            : None,     # Optimizer -> default ist Adam
            'bridge_learning_rate'        : 0.0001,   # learningrate
            'bridge_gamma'                : 0.9,      # discount_factor for reward
            'bridge_initial_population'   : 1000,     # initial number of experiences in buffer
            'bridge_batch_size'           : 32,       # batch_size for training
  
            # loop control  
            'loop_run_name'               : "base",   # runname for logfile
            'loop_bound_avg_reward'       : 1000.0,   # target avg reward
            'loop_logtb'                  : True,     # Log to Tensorboard Logfile
    })
}
    

In [ ]:
def create_control(params: SimpleNamespace) -> LoopControl:
    env = CarEnv()
    
    agent = Agent(env, devicestr       = params.agent_device, 
                       gamma           = params.agent_gamma_exp, 
                       buffer_size     = params.agent_buffer_size,
                       eps_start       = params.agent_eps_start,
                       eps_final       = params.agent_eps_final,
                       eps_frames      = params.agent_eps_frames,
                       target_net_sync = params.agent_target_net_sync)
    
    bridge = Bridge(agent=agent,
                    optimizer          = params.bridge_optimizer,
                    learning_rate      = params.bridge_learning_rate,
                    gamma              = params.bridge_gamma,
                    initial_population = params.bridge_initial_population,
                    batch_size         = params.bridge_batch_size,
                   )
    
    control = LoopControl(
                   bridge              = bridge, 
                   run_name            = params.loop_run_name, 
                   bound_avg_reward    = params.loop_bound_avg_reward,
                   logtb               = params.loop_logtb)
    
    return control

In [ ]:
def run_example(config_name: str):
    # get rid of missing metrics warning
    warnings.simplefilter("ignore", category=UserWarning)
    
    control = create_control(HYPERPARAMS[config_name])
    control.run()

In [ ]:
run_example('base_setup')

Current run is terminating due to exception: to() received an invalid combination of arguments - got (type), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)
.
Engine run is terminating due to exception: to() received an invalid combination of arguments - got (type), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)
.


TypeError: to() received an invalid combination of arguments - got (type), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)
